# Project Types of Interest

## Looking into Organizations with the following project types:
*  purchasing wheelchair accessible vehicle
* on-demand scheduling 
* dispatching software platforms

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from siuba import *
import numpy as np

import altair as alt

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

from calitp import to_snakecase

from IPython.display import display, Markdown, HTML

In [3]:
df = pd.read_csv('5310_info_pt.csv')

In [4]:
grouped =pd.read_csv('gs://calitp-analytics-data/data-analyses/5310/5310_org_info_manualfill.csv')

In [5]:
def labeling(word):
    LABEL_DICT = {
        "dist": "District",
        "nunique": "Number of Unique",
        "n":"Count",
        "name_NTD_Airtable":"Organization Name"
    }

    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        #word = word.replace("n_", "Number of ").title()
        word = word.replace("unique_", "Number of Unique ").title()
        word = word.replace("_", " ").title()

    return word

def bar_chart(df, x_col, y_col, color_col, chart_title=""):

    bar = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(x_col, title=labeling(x_col), sort=("-y")),
            y=alt.Y(y_col, title=labeling(y_col)),
            color=alt.Color(
                color_col,
                scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                legend=alt.Legend(title=(labeling(color_col)), symbolLimit=10)
            )
        ).properties(title=chart_title))
    
    chart = styleguide.preset_chart_config(bar)
    
    return chart

In [6]:
org_counts= grouped>>count(_.organization_type)
bar_chart(org_counts,
    "organization_type",
    "n",
    "organization_type",
    "5310 Organization Types"
)

alt.Chart(...)

In [7]:
single_county = df[~df["county"].str.contains(",", na=False)]
bar_chart((single_county>>count(_.county)),
    "county",
    "n",
    "county",
    "Counties with 5310 Organizations (Single County)"
)

alt.Chart(...)

In [8]:
county_explode = df>>select(_.organization_name, _.ntd_id, _.itp_id, _.organization_type, _.city, _.county, _.merge_status)
county_explode.explode('county')

county_explode2 = county_explode.assign(county_split = df['county'].str.split(', ')).drop("county", axis=1)\
    .rename(columns = {"county_split": "county"})
county_explode2 = county_explode2.explode('county')

bar_chart((county_explode2>>count(_.county)),
    "county",
    "n",
    "county",
    "Counties with 5310 Organizations Operating"
)

alt.Chart(...)

## Project Types

In [9]:
#df.sample()

In [10]:
ptype_counts = (df
 >>group_by(_.organization_name)
 >>summarize(n_vp = _.vehicle_purchase.sum(), 
            n_oa = _.operating_assistance.sum(),
            n_mm = _.mobility_management.sum(),
            n_hsp = _.hardware_software_purchase.sum(),
            n_c = _.communications.sum(),
            n_fe = _.facilities_eqp.sum(),
             n_s = _.surveillance.sum(),
             n_sub = _.subsidy.sum()
            )
)

In [11]:
#ptype_counts.sample()

In [12]:
# columns = ['n_vp','n_oa','n_mm','n_hsp','n_c','n_fe','n_s','n_sub']

# for column in columns: 
#     ptype_counts.column[ptype_counts[column]> 0] = 1

In [13]:
#WORKED for getting yes/no present
## from https://stackoverflow.com/questions/61996932/replacing-values-greater-1-in-a-large-pandas-dataframe

#ptype_counts.set_index('organization_name',inplace=True)
#ptype_counts[ptype_counts >= 1] = 'Yes'
#ptype_counts.reset_index()

In [14]:
#ptype_counts = ptype_counts.replace(0, 'No')

In [15]:
#ptype_counts.to_csv('single_counts_2.csv')

In [16]:
# # def project_present(df):
# #     columns = ['n_vp','n_oa','n_mm','n_hsp','n_c','n_fe','n_s','n_sub']
# #     if df.column != 0:
# #         return 1
# #     else:
# #         return 0

# # for column in columns:
# #     df[f"{column}_2"] = df.apply(lambda x: project_present(x), axis=1)
    
    
    
# columns = ['n_vp','n_oa','n_mm','n_hsp','n_c','n_fe','n_s','n_sub']

# for column in columns:
#     ptype_counts[f"{column}_2"] = ptype_counts[column].any(1).astype(int)


In [17]:
ptype_counts.set_index('organization_name',inplace=True)
ptype_counts[ptype_counts >= 1] = 1
ptype_counts = ptype_counts.reset_index()

In [18]:
ptype_counts = ptype_counts.rename(
    columns={
        "organization_name": "Organization Name",
        "n_vp": "Vehicle Purchase",
        "n_oa": "Operating Assistance",
        "n_mm": "Mobility Management",
        "n_hsp": "Hardware/Software Purchase",
        "n_c": "Communications",
        "n_fe": "Facilities Equipment",
        "n_s": "Surveillance",
        "n_sub": "Subsidies",
    }
)

In [19]:
sum_ptype = ((ptype_counts)
               .agg({'Vehicle Purchase':'sum', 'Operating Assistance':'sum',
                      'Mobility Management':'sum','Hardware/Software Purchase':'sum','Communications':'sum',
                     'Facilities Equipment':'sum', 'Surveillance':'sum', 'Subsidies':'sum'}).reset_index())
sum_ptype = sum_ptype.rename(columns = {"index":"Project Types",0:"Count"})
bar_chart((sum_ptype),
    "Project Types",
    "Count",
    "Project Types",
    "Number of Organizations using this Project Type"
)

alt.Chart(...)

In [20]:
columns = [
    "Vehicle Purchase",
    "Operating Assistance",
    "Mobility Management",
    "Hardware/Software Purchase",
    "Communications",
    "Facilities Equipment",
    "Surveillance",
    "Subsidies",
]
for column in columns:
    org_counts = (ptype_counts 
                  >> filter(_[column] == 1)
                  >> count(_['Organization Name'])
                  >> select(_['Organization Name']))
    display(HTML(f"<strong>Organization with {column} Project Types Present</strong>"))
    display(HTML(org_counts.to_html(index=False)))

Organization Name
ARC Bakersfield
ARC Imperial Valley
Able Industries
Alegria Community Living
Angel View Inc
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services
Avenidas
Bay Area Outreach & Recreation Program Inc
Big Valley 50 Plus
Butte County Association of Governments Butte Regional Transit


Organization Name
ARC Imperial Valley
Amador Transit
Angel View Inc
Area 1 Agency on Aging
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services
Avenidas
Catholic Charities of the Diocese of Stockton
Choice in Aging
City of Lafayette: Lamorinda Sprit Van Program
City of Rio Vista


Organization Name
Amador Transit
Catholic Charities of the Diocese of Stockton
City of Roseville
City of Visalia
Common Ground Senior Services
Contra Costa Transportation Authority (CCTA)
County of Sonoma Human Services Department Adult and Aging Division
Eden I&R INC
Friends of Children with Special Needs
Full Access & Coordinated Transportation Inc


Organization Name
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services
Catholic Charities of the Diocese of Stockton
Community Bridges Liftline
County of Sonoma Human Services Department Adult and Aging Division
Foothill AIDS Project
Milestones of Development INC
Porterville Sheltered Workshop
San Joaquin Regional Transit District
Sunline Transit Agency
United Cerebral Palsy Association of Greater Sacramento Inc


Organization Name
Center for Elders' Independence
Community Bridges Liftline
Eastern Sierra Transit Authority
Humboldt Senior Resource Center
North of the River Recreation and Park District
Porterville Sheltered Workshop
Self-Help for the Elderly
United Cerebral Palsy Association of Greater Sacramento Inc
Work Training Center


Organization Name
Eastern Sierra Transit Authority
Lighthouse for the Blind and Visually Impaired
Milestones of Development INC
North and South of Market Adult Day Health
Porterville Sheltered Workshop


Organization Name
Community Bridges Liftline
Institute on Aging
Porterville Sheltered Workshop
United Cerebral Palsy Association of Greater Sacramento Inc


Organization Name
San Mateo County Transit District


### Purchasing Wheelchair Accessible Vehicles

In [21]:
# no project descirption matches for `wheelchair`, `lifts`, `ramps`, and `securement` 

In [22]:
# display(
#     HTML(
#         (f"There are {len(df>>filter(_.description.str.contains('wheelchair')))} project descriptions that mention Wheelchair")
#     ))
    

In [23]:
display(
    HTML(f"Of the {len(df>>filter(_.vehicle_purchase==1)>>count(_.organization_name))} agencies using 5310 funds "
      f"for vehicle purchases, "
      f"{len(df>>filter(_.vehicle_purchase==1)>>filter(_.description.str.contains('Bus'))>>count(_.organization_name))} "
      f"are for Bus Purchases, "
      f"and {len(df>>filter(_.vehicle_purchase==1)>>filter(_.description.str.contains('Van'))>>count(_.organization_name))} "
      f"are for Van Purchases"
         f"<br> There is {len(df>>filter(_.description.str.contains('wheelchair')))} mention of wheelchair "
         f"accessible vehicles, ramps or other accessiblity features"
        ))

**Organizations using funds for Vehicle Purchase**

In [24]:
vehicle_purchase= (df>>filter(_.vehicle_purchase==1)>>count(_.organization_name)>>arrange(-_.n)).head(10)
vehicle_purchase = vehicle_purchase.rename(columns={"organization_name":"Organization Name",
                                "n":"Counts"})
display(HTML(vehicle_purchase.to_html(index=False)))

Organization Name,Counts
On Lok Senior Health Services,50
Pride Industries One Inc,49
United Cerebral Palsy of San Luis Obispo County,46
United Cerebral Palsy Association of Greater Sacramento Inc,42
Kings Rehabilitation Center,30
Monterey-Salinas Transit,26
North of the River Recreation and Park District,25
ARC Bakersfield,20
Vivalon Inc,20
Butte County Association of Governments Butte Regional Transit,19


### On-demand Scheduling

**Organizations with Operating Assistance Project types**

In [25]:
display(HTML(f"There is {len(df>>filter(_.description.str.contains('demand')))} mention of Demand related project types"))
display(HTML(f"There are {len((df>>filter(_.operating_assistance==1)>>count(_.organization_name)>>arrange(-_.n)))} "
             f"mentions of Operating Assistance"))

In [26]:
operating_assistance= (df>>filter(_.operating_assistance==1)>>count(_.organization_name)>>arrange(-_.n)).head(10)
operating_assistance = operating_assistance.rename(columns={"organization_name":"Organization Name",
                                "n":"Counts"})
display(HTML(operating_assistance.to_html(index=False)))

Organization Name,Counts
Independent Living Partnership,8
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,6
ARC Imperial Valley,4
Choice in Aging,4
County of Sonoma Human Services Department Adult and Aging Division,4
Mobility Matters,4
Angel View Inc,3
Catholic Charities of the Diocese of Stockton,3
City of Lafayette: Lamorinda Sprit Van Program,3
County of Sacramento Department of Child Family and Adult Services,3


### Dispatching Software Platforms

In [27]:
display(
    HTML(f"Out of the {len(df>>filter(_.hardware_software_purchase==1)>>count(_.organization_name))} "
         f"with harware or software purchases, "
         f"{len(df>>filter(_.hardware_software_purchase==1)>>filter(_.description.str.contains('Software'))>>count(_.organization_name))}" 
             f"organization used 5310 funds for software purposes"))
             

In [28]:
software = (df>>filter(_.hardware_software_purchase==1)>>filter(_.description.str.contains('Software'))>>count(_.organization_name))
software = software.rename(columns={"organization_name":"Organization Name",
                                "n":"Counts"})
display(HTML(software.to_html(index=False)))


Organization Name,Counts
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2
Catholic Charities of the Diocese of Stockton,2
County of Sonoma Human Services Department Adult and Aging Division,1
Yolo County Transportation District,2


## Project Type Funding Analysis

### By Organization & Year

**Sum of Funds Allocated to Organizations by Year (Top 20)**

In [29]:
top20_sum = (
    df
    >> group_by(_.organization_name, _.project_year)
    >> summarize(sum_allocated_by_year=_.allocationamount.sum())
    >> arrange(-_.sum_allocated_by_year)
).head(20)
top20_sum = top20_sum.rename(columns={"organization_name":"Organization Name",
                                      "project_year":"Project Year",
                                      "sum_allocated_by_year":"Sum Allocated by Year"}) 
top20_sum = (top20_sum.style.hide(axis='index').format(formatter={("Sum Allocated by Year"): "${:,.2f}"}))

display(top20_sum)
#display(HTML(top20_sum.to_html(index=False)))

Organization Name,Project Year,Sum Allocated by Year
United Cerebral Palsy Association of Greater Sacramento Inc,2017,"$1,782,550.00"
On Lok Senior Health Services,2019,"$1,480,000.00"
Pride Industries One Inc,2019,"$1,257,714.00"
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2017,"$1,246,128.00"
Mobility Matters,2017,"$1,160,438.00"
Friends of Children with Special Needs,2019,"$1,152,937.00"
Pride Industries One Inc,2017,"$1,135,727.52"
Independent Living Partnership,2017,"$1,114,031.00"
ARC Bakersfield,2017,"$1,008,340.28"
Marin County Transit District,2017,"$1,000,000.00"


### By Organization & Project Types

**Vehicle Purchases**

In [30]:
vp = (df
 >>filter(_.vehicle_purchase==1)
 >> group_by(_.organization_name, _.project_year)
 >> summarize(sum_allocated = _.allocationamount.sum())
  >> arrange(-_.sum_allocated)
).head(20)

vp = (vp.style.hide(axis='index').format(formatter={("sum_allocated"): "${:,.2f}"}))
display(vp)

organization_name,project_year,sum_allocated
On Lok Senior Health Services,2019,"$1,480,000.00"
United Cerebral Palsy Association of Greater Sacramento Inc,2017,"$1,449,680.00"
Pride Industries One Inc,2019,"$1,209,000.00"
Pride Industries One Inc,2017,"$1,135,727.52"
ARC Bakersfield,2017,"$1,008,340.28"
Monterey-Salinas Transit,2017,"$980,000.00"
United Cerebral Palsy of San Luis Obispo County,2017,"$634,000.00"
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2017,"$619,600.00"
El Dorado County Transit Authority,2017,"$610,000.00"
Institute on Aging,2017,"$560,000.00"


**Operating Assistance**

In [31]:
oa= (df
 >>filter(_.operating_assistance==1)
 >> group_by(_.organization_name, _.project_year)
 >> summarize(sum_allocated = _.allocationamount.sum())
 >> arrange(-_.sum_allocated)
).head(20)

oa = (oa.style.hide(axis='index').format(formatter={("sum_allocated"): "${:,.2f}"}))
display(oa)

organization_name,project_year,sum_allocated
Independent Living Partnership,2017,"$1,114,031.00"
LIFE ElderCare,2017,"$802,658.00"
Catholic Charities of the Diocese of Stockton,2017,"$721,986.00"
Town of Truckee,2017,"$702,506.00"
Marin County Transit District,2017,"$700,000.00"
West Valley Community Services,2019,"$649,453.00"
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2017,"$624,048.00"
Dignity Health Connected Living,2017,"$530,302.00"
Community Bridges Liftline,2017,"$475,200.00"
United Cerebral Palsy Association of Greater Sacramento Inc,2019,"$467,625.00"


**Software Purchase**

In [32]:
sp = (df
 >>filter(_.hardware_software_purchase==1)
 >>filter(_.description.str.contains('Software'))
 >> group_by(_.organization_name, _.project_year)
 >> summarize(sum_allocated = _.allocationamount.sum())
  >> arrange(-_.sum_allocated)
).head(20)

sp = (sp.style.hide(axis='index').format(formatter={("sum_allocated"): "${:,.2f}"}))
display(sp)


organization_name,project_year,sum_allocated
Catholic Charities of the Diocese of Stockton,2019,"$20,000.00"
County of Sonoma Human Services Department Adult and Aging Division,2018,"$10,000.00"
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2017,$0.00
Yolo County Transportation District,2017,$0.00


In [33]:
hsp = (df
 >>filter(_.hardware_software_purchase==1)
 >> group_by(_.organization_name, _.project_year)
 >> summarize(sum_allocated = _.allocationamount.sum())
       >> arrange(-_.sum_allocated)
).head(20)

hsp = (hsp.style.hide(axis='index').format(formatter={("sum_allocated"): "${:,.2f}"}))
display(hsp)


organization_name,project_year,sum_allocated
United Cerebral Palsy Association of Greater Sacramento Inc,2017,"$61,600.00"
Catholic Charities of the Diocese of Stockton,2019,"$28,500.00"
Community Bridges Liftline,2017,"$17,196.00"
County of Sonoma Human Services Department Adult and Aging Division,2018,"$10,000.00"
Community Bridges Liftline,2019,"$7,954.00"
Porterville Sheltered Workshop,2019,"$7,742.00"
San Joaquin Regional Transit District,2017,"$7,571.00"
Sunline Transit Agency,2019,"$5,150.00"
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2017,"$2,480.00"
Milestones of Development INC,2019,"$1,400.00"


### By County

In [34]:
county = (df
 >> group_by(_.county)
 >> summarize(sum_allocated_by_county = _.allocationamount.sum())
)
county = (county.style.hide(axis='index').format(formatter={("sum_allocated_by_county"): "${:,.2f}"}))
display(county)

county,sum_allocated_by_county
Alameda,"$2,982,664.00"
"Alameda, Contra Costa","$671,900.00"
Amador,"$469,995.00"
"Amador, Calaveras","$299,479.40"
Butte,"$1,235,790.10"
"Butte, Glen","$813,194.00"
Calaveras,$0.00
"Calaveras, Amador, Tuolumne","$826,157.00"
Colusa,"$162,400.00"
Contra Costa,"$3,007,198.00"
